# Titanic - Machine Learning from Disaster

part 1 - data cleaning

https://dataisutopia.com/blog/preprocessing-titanic-dataset/

https://medium.com/analytics-vidhya/exploratory-data-analysis-of-titanic-survival-problem-e3af0fb1f276

https://towardsdatascience.com/machine-learning-with-the-titanic-dataset-7f6909e58280

In [ ]:
#load packages
import re
import warnings

import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
#get data
df_train = pd.read_csv('Data/train.csv')
df_test = pd.read_csv('Data/test.csv')

## Data inspection

In [ ]:
print("Train data contains " + str(len(df_train)) + " rows and " + str(len(df_train.columns)) + " columns")
print("Test data contains " + str(len(df_test)) + " rows and " + str(len(df_test.columns)) + " columns")

In [ ]:
df_train.head(5)

In [ ]:
df_test.head(5)

In [ ]:
print("\nMissing values of train dataset:\n") 
display(df_train.isnull().sum()) 
print("\nMissing values of test dataset:\n") 
display(df_test.isnull().sum()) 

### Some additional info

In [ ]:
print("\nGeneral information of train data:\n") 
df_train.info()

In [ ]:
print("\nDescriptive statistics of numeric columns in train data:\n") 
df_train.describe().round(2) 

In [ ]:
#visualise the missing values
sns.heatmap(df_train.isnull(), cmap='viridis', cbar=False)

## Data cleaning

In [ ]:
#create a complete dataset
df_all = pd.concat([df_train, df_test], sort=True).reset_index(drop=True)

### Inspect missing values

In [ ]:
#sumarize missing ages
print('Number of passenger without age: ' + str(df_all.Age.isnull().sum()))
print('This is ' + str(round(df_all.Age.isnull().sum()/len(df_all)*100,0)) + '% of the total passengers')

In [ ]:
#check median age per category
print("Median per Class and Sex:")
display(df_all.groupby(['Pclass', 'Sex'])['Age'].median())

#check if there are enough samples
print("Counts per Class and Sex:")
display(df_all.groupby(['Pclass', 'Sex'])['Age'].count())

In [ ]:
#find row with missing Fare is used to search for similar passengers
df_all.loc[df_all['Fare'].isnull()]

In [ ]:
#these passenger were together and Embarked in Southampton (source https://www.encyclopedia-titanica.org/)
df_all.loc[df_all['Embarked'].isnull()]

In [ ]:
#count missing values in Cabin column
print('There are ' + str(df_all['Cabin'].isnull().sum()) + ' cabin values missing!\n')

#too much values are missing, check for additional features
print('There are ' + str(len(df_all['Cabin'].unique())) + ' unique cabin values:')
print(str(df_all['Cabin'].unique()))

### Create a cleaned dataset

In [ ]:
def cleaning(df):
    dfc = df.copy()

    #drop PassengerId column because it is useless
    dfc.drop('PassengerId', axis=1, inplace=True)

    #fill gaps in Age column with random values around mean age
    # age_mean = dfc['Age'].mean()
    # age_std = dfc['Age'].std()

    # def fill_age(col):
    #     if np.isnan(col):
    #         return np.random.randint(age_mean-age_std, age_mean+age_std)
    #     return(col)
    
    # dfc['Age'] = dfc['Age'].apply(fill_age).astype(int)

    #a better method is to fill gaps with median per group of class and sex
    dfc['Age'] = dfc.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
    
    #fill few missing values in Embarked with most frequent value (Southampton)
    dfc['Embarked'].fillna('S', inplace=True)

    #fill missing Fare value of Mr. Thomas with median of similar passengers
    SimPasFare = dfc.loc[(dfc['Pclass'] == 3) & (dfc['SibSp'] == 0) & (dfc['Embarked'] == 'S')]['Fare'].median()
    dfc.loc[dfc['Fare'].isnull(), 'Fare'] = SimPasFare

    #round fare and age column
    dfc['Age'] = dfc['Age'].round(0).astype(int)
    dfc['Fare'] = dfc['Fare'].round(0).astype(int)

    return dfc

In [ ]:
df_all_clean = cleaning(df_all)

## Feature engineering

In [ ]:
#make copy of cleaned dataset
df_all_clean_feat = df_all_clean.copy()

In [ ]:
#add children as addition to male/female
def children(passenger):
    age, sex = passenger

    if age <16:
        return 'child'
    else:
        return sex

df_all_clean_feat['Person'] = df_all_clean_feat[['Age','Sex']].apply(children, axis=1)

In [ ]:
#add column about if passenger had a cabin
def missing_cabin(col):

    if isinstance(col, type(np.nan)):
        return 0
    return 1

df_all_clean_feat['HasCabin'] = df_all_clean_feat['Cabin'].apply(missing_cabin)

In [ ]:
def family (df):

    df.is_copy = False

    #add column about traveling alone
    df['IsAlone'] = df['Parch'] + df['SibSp']
    df['IsAlone'] = np.where(df['IsAlone']>0, 0, 1)

    #add column about family size
    df['FamilySize'] = df['Parch'] + df['SibSp'] + 1

    #bin the family size
    df['FamilySizeBin'] = df['FamilySize'].apply(lambda x: 1 if x==1 else (2 if x==2 else (3 if (x==3) | (x==4) else (4 if x >= 5 else 0))))

    return df

df_all_clean_feat = family(df_all_clean_feat)

In [ ]:
#there are some people who paid a lot of money
df_all_clean_feat.boxplot(column=['Fare'])

In [ ]:
#divide the fare in 4 groups to deal with the outliers
def group_fare (df, colname):
    return pd.qcut(df[colname], 5, labels = [1, 2, 3, 4, 5]).astype(int)

df_all_clean_feat['CategoricalFare'] = group_fare(df_all_clean_feat, 'Fare')

In [ ]:
#number of passenger per group are more or less equal (qcut method)
df_all_clean_feat['CategoricalFare'].value_counts()

In [ ]:
#there were some old people on the ship 
df_all_clean_feat.boxplot(column=['Age'])

In [ ]:
#divide age in 5 groups to deal with the outliers
df_all_clean_feat['CategoricalAge'] = pd.cut(df_all_clean_feat['Age'].astype(int), 5)

In [ ]:
#number of passenger per group are different (cut method)
df_all_clean_feat['CategoricalAge'].value_counts()

In [ ]:
#add column with ticket frequency
df_all_clean_feat['TicketFrequency'] = df_all_clean_feat.groupby('Ticket')['Ticket'].transform('count')

In [ ]:
#add columns with deck information
df_all_clean_feat['Deck'] = df_all_clean_feat['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'M')

#only one passenger on deck T (boat deck), so replace with deck A
idx = df_all_clean_feat[df_all_clean_feat['Deck'] == 'T'].index
df_all_clean_feat.loc[idx, 'Deck'] = 'A'

#group several deck according to classes
df_all_clean_feat['DeckGroup'] = df_all_clean_feat['Deck']
df_all_clean_feat['DeckGroup'] = df_all_clean_feat['DeckGroup'].replace(['A', 'B', 'C'], 'ABC')
df_all_clean_feat['DeckGroup'] = df_all_clean_feat['DeckGroup'].replace(['D', 'E'], 'DE')
df_all_clean_feat['DeckGroup'] = df_all_clean_feat['DeckGroup'].replace(['F', 'G'], 'FG')

In [ ]:
#create column with title of the passenger
def title (df, colname):

    def find_title (x):
        title_search = re.search('([A-Za-z]+)\.', x)

        if title_search:
            title = title_search.group(1)

            if title in ['Mlle', 'Ms', 'Miss']:
                return 'Miss'
            elif title in ['Mme', 'Mrs']:
                return 'Mrs'
            elif title in ['Mr', 'Master']:
                return 'Mr'
            else:
                return 'Rare'

        return ""

    return_title = df[colname].apply(find_title)

    return return_title

df_all_clean_feat['Title'] = title(df_all_clean_feat, 'Name')         

In [ ]:
#add column with info about marriage
df_all_clean_feat['IsMarried'] = 0
df_all_clean_feat['IsMarried'].loc[df_all_clean_feat['Title'] == 'Mrs'] = 1

## Data encoding

In [ ]:
#make copy of engineered dataset
df_all_clean_feat_enc = df_all_clean_feat.copy()

#list of all columns
df_all_clean_feat_enc.columns

### One-Hot Encoding

In [ ]:
def encodingOne (df):
    
    #select cols to do the encoding
    cols = ['Person', 'Embarked', 'Title', 'DeckGroup', 'CategoricalAge', 'CategoricalFare', 'FamilySizeBin', 'Pclass']

    #keep the orginal columns
    df_orginal = df[cols]

    #do the encoding
    df_dummies = pd.get_dummies(df, columns=cols)

    #append the orginal columns
    df = pd.concat([df_dummies, df_orginal], axis=1)

    return df

df_all_clean_feat_enc = encodingOne(df_all_clean_feat_enc)

### Integer Encoding

In [ ]:
feat_list = ['Embarked', 'Person', 'Title', 'Sex', 'Deck', 'DeckGroup', 'CategoricalAge']

for feat in feat_list:
    df_all_clean_feat_enc[feat] = LabelEncoder().fit_transform(df_all_clean_feat_enc[feat])

## Final data

In [ ]:
#select only columns needed
drop_list = ['Cabin', 'Ticket', 'Name']
df_all_final = df_all_clean_feat_enc.drop(drop_list, axis=1)

In [ ]:
#double check for missing values
df_all_final.loc[:, df_all_final.columns != 'Survived'].isnull().to_numpy().any()

In [ ]:
df_all_final.head()

In [ ]:
df_all_final.to_csv('Data/clean.csv', index=False)